In [1]:
import os #file system operations
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.image as impl
from sklearn.metrics import roc_curve, classification_report, accuracy_score
from scipy.ndimage import generic_filter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
if not os.path.exists('/content/drive/MyDrive/ML_IITG/Unimodel_background_model/noise_removed_bg'):
  os.makedirs('/content/drive/MyDrive/ML_IITG/Unimodel_background_model/noise_removed_bg')

In [7]:
RGB_path = '/content/drive/MyDrive/ML_IITG/Unimodel_background_model/Images'
ground_truth_path = '/content/drive/MyDrive/ML_IITG/Unimodel_background_model/BakSubGroundTruth'

1. Several lambda values which we'll provide
2. iterate over lambda which controls the sensitivity of foreground detection

In [9]:
lambda_val = np.array([0.5]) #,1.0,2.0,3.0,4.0,5.0,6.0,7.0])
plt.figure(figsize=(8,6))

for l in lambda_val:
  print(f'\n-- Processing for lambda = {l} :')
  start_time = time.time()

  try:
    files = sorted(os.listdir(RGB_path))
    if not files:
      print(f"Error: The directory '{RGB_path}' is empty or not found")
      print("Please ensure that dataset has been uploaded at the right location")
  except FileNotFoundError:
    print(f"Error: The directory '{RGB_path}' was not found")
    print("Please ensure that dataset has been uploaded at the right location")
    break


  #Initialization of background model

  v_d = 9.0 #Default variance
  m = 5 #size of the neighborhood window for noise removal (5*5 pixel square)
  eta = 0.6 #Threshold for noise removal
  t_c = 100 #time constant for model update
  t = 0

  #Mean of the background model
  first_image_path = os.path.join(RGB_path,files[0])
  M1 = impl.imread(first_image_path).astype(np.float64)
  rows, columns, channels = M1.shape

  #V1 : Variance of the background model
  V1 = np.full_like(M1, v_d, dtype=np.float64)


  Y_actual_flat = []
  Y_predicted_flat = []
  lambda_sq = l**2

  for img_file in files:
    img_path = os.path.join(RGB_path,img_file)
    image = impl.imread(img_path).astype(np.float64)
    t = t+1

    #Foreground detection
    distance = image - M1
    is_background_per_channel = (distance**2) <= (lambda_sq * V1)

    #Pixel is background (True) only if its background in all the channels
    is_background_mask = np.all(is_background_per_channel, axis=2)

    #Convert boolean mask to binary values: Foreground = 255, Background = 0
    foreground_mask_initial = np.where(is_background_mask, 0, 255).astype(np.uint8)

    #Noise Removal (Optimized)
    def filter_func(neighborhood):
      return 1 if np.count_nonzero(neighborhood) / (m*m) >= eta else 0

    #Normalise the images to 0 and 1 for easy counting
    normalised_mask = foreground_mask_initial/255.0
    filtered_mask_normalised = generic_filter(normalised_mask,filter_func,size=m)


    #Convert back to 0 and 255
    noise_removed_img = (filtered_mask_normalised*255).astype(np.uint8)
    plt.imsave(f"/content/drive/MyDrive/ML_IITG/Unimodel_background_model/noise_removed/{l}_{img_file}",noise_removed_img, cmap='gray')

    #Background Model Update

    #Create a mask where background pixels are True
    update_mask = (noise_removed_img == 0)

    #reshape to broadcast across the 3 color channels
    update_mask_3d = np.stack([update_mask]*channels,axis=-1)

    #calculate learning rate alpha_t
    alpha = 1.0/t if t <= t_c else 1.0/t_c

    #Update mean M1
    M1[update_mask_3d] += (alpha*distance[update_mask_3d])

    #Update Variance V1
    term1 = (1-alpha)*V1[update_mask_3d]
    term2 = alpha*(distance[update_mask_3d]**2)
    V1[update_mask_3d] = term1+term2

    # Evaluation
    try:
      g_file = 'groundMono'+img_file.split('_')[1]
      ground_truth_path_full = os.path.join(ground_truth_path,g_file)
      y_true = cv2.imread(ground_truth_path_full,0)

      #Normalize to 0 (bg) and 1 (fg)
      y_true = np.where(y_true==255,1,0)
      y_predict = np.where(noise_removed_img ==255,1,0)

      Y_actual_flat.extend(y_true.ravel())
      Y_predicted_flat.extend(y_predict.ravel())

    except (FileNotFoundError, IndexError):
      print(f"Could not find or parse ground truth for {img_file}")
      continue

    #Reporting

    if not Y_actual_flat or not Y_predicted_flat:
      print("No ground truth was processed. Skipping the metrics")
      continue

    fpr, tpr, thresholds = roc_curve(Y_actual_flat,Y_predicted_flat)
    plt.plot(fpr,tpr, label = f'lambda = {l}')

    print(f"Total time = {(time.time() - start_time)} seconds")
    print(classification_report(Y_actual_flat,Y_predicted_flat))



-- Processing for lambda = 0.5 :


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ML_IITG/Unimodel_background_model/noise_removed/0.5_AirstripRunAroundFeb2006_1300.bmp'

<Figure size 800x600 with 0 Axes>